## Load Data
The data can be retrived from github.

In [1]:
!wget https://raw.githubusercontent.com/AgencyPMG/ProgrammingChallenges/master/sql-assessment/marketing_data.csv

--2022-12-16 07:20:47--  https://raw.githubusercontent.com/AgencyPMG/ProgrammingChallenges/master/sql-assessment/marketing_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 655 [text/plain]
Saving to: ‘marketing_data.csv’

marketing_data.csv  100%[===================>]     655  --.-KB/s    in 0s      

2022-12-16 07:20:47 (23.9 MB/s) - ‘marketing_data.csv’ saved [655/655]



In [2]:
!wget https://raw.githubusercontent.com/AgencyPMG/ProgrammingChallenges/master/sql-assessment/store_revenue.csv

--2022-12-16 07:20:48--  https://raw.githubusercontent.com/AgencyPMG/ProgrammingChallenges/master/sql-assessment/store_revenue.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1372 (1.3K) [text/plain]
Saving to: ‘store_revenue.csv’

store_revenue.csv   100%[===================>]   1.34K  --.-KB/s    in 0s      

2022-12-16 07:20:48 (43.4 MB/s) - ‘store_revenue.csv’ saved [1372/1372]



## Import CSV into SQLite

Prior to working with the data in SQL, we will need to import it into the database. We achieve this by reading the data first into Python using `pandas` and, then, using the `.to_sql()` export handler to write the data as a table into the database.

In [3]:
import pandas as pd 

df1 = pd.read_csv("marketing_data.csv")
df2 = pd.read_csv("store_revenue.csv")

In [4]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///assessment.db', echo=False)

# Create the X table in the sqlite database
df1.to_sql('marketing_data', index = False, if_exists = 'replace', con=engine)

# Create the Y table in the sqlite database
df2.to_sql('store_revenue', index = False, if_exists = 'replace', con=engine)

In [5]:
%load_ext sql
%sql sqlite:///assessment.db

'Connected: @assessment.db'

## Question 0
(Already done for you as an example) Select the first 2 rows from the marketing data​

In [6]:
%%sql

select * from marketing_data limit 2;

 * sqlite:///assessment.db
Done.


date,geo,impressions,clicks
2016-01-01,TX,2532,45
2016-01-01,CA,3425,63


## Question 1

Generate a query to get the sum of the clicks of the marketing data​

In [7]:
%%sql

SELECT SUM(clicks) AS sum_clicks
FROM marketing_data

 * sqlite:///assessment.db
Done.


sum_clicks
1792


## Question 2 
Generate a query to gather the sum of revenue by store_location from the store_revenue table​

In [8]:
%%sql

SELECT store_location, SUM(revenue) AS sum_revenue
FROM store_revenue
GROUP BY store_location

 * sqlite:///assessment.db
Done.


store_location,sum_revenue
United States-CA,235237
United States-NY,51984
United States-TX,9629


## Question 3 
Merge these two datasets so we can see impressions, clicks, and revenue together by date and geo. Please ensure all records from each table are accounted for.​

Firstly, we need to check the columns in those two tables.

In [9]:
%%sql

SELECT * FROM marketing_data LIMIT 5

 * sqlite:///assessment.db
Done.


date,geo,impressions,clicks
2016-01-01,TX,2532,45
2016-01-01,CA,3425,63
2016-01-01,NY,3532,25
2016-01-01,MN,1342,784
2016-01-02,TX,3643,23


In [10]:
%%sql

SELECT * FROM store_revenue LIMIT 5

 * sqlite:///assessment.db
Done.


date,brand_id,store_location,revenue
2016-01-01,1,United States-CA,100
2016-01-01,1,United States-TX,420
2016-01-01,1,United States-NY,142
2016-01-02,1,United States-CA,231
2016-01-02,1,United States-TX,2342


Next, we noticed that the store_location in `store_revenue` is recorded differently from geo in `marketing_data`. In this case, we used common table expression and case when clause to convert the store_location into the same notation with geo.

In [11]:
%%sql

SELECT DISTINCT store_location FROM store_revenue

 * sqlite:///assessment.db
Done.


store_location
United States-CA
United States-TX
United States-NY


In [12]:
%%sql

WITH cte AS(
  SELECT date, brand_id, revenue, (CASE store_location 
  WHEN 'United States-CA' THEN 'CA'
  WHEN 'United States-TX' THEN 'TX'
  ELSE 'NY'
  END) AS geo
  FROM store_revenue
)

SELECT * FROM cte LIMIT 5

 * sqlite:///assessment.db
Done.


date,brand_id,revenue,geo
2016-01-01,1,100,CA
2016-01-01,1,420,TX
2016-01-01,1,142,NY
2016-01-02,1,231,CA
2016-01-02,1,2342,TX


Then, since we want to ensure all records from each table are included, we would like to use full outer join. However, full outer join and right join are not currently supported in sqlite. Therefore, we used union and left join.

In [13]:
%%sql

WITH cte AS(
  SELECT date, brand_id, revenue, (CASE store_location 
  WHEN 'United States-CA' THEN 'CA'
  WHEN 'United States-TX' THEN 'TX'
  ELSE 'NY'
  END) AS geo
  FROM store_revenue
)

SELECT c.date, c.geo, brand_id, impressions, clicks, revenue
FROM cte c
LEFT JOIN marketing_data m
ON c.date = m.date AND c.geo = m.geo

UNION

SELECT m.date, m.geo, brand_id, impressions, clicks, revenue
FROM marketing_data m
LEFT JOIN cte c
ON c.date = m.date AND c.geo = m.geo

 * sqlite:///assessment.db
Done.


date,geo,brand_id,impressions,clicks,revenue
2016-01-01,CA,1,3425,63,100
2016-01-01,CA,2,3425,63,234
2016-01-01,MN,None,1342,784,None
2016-01-01,NY,1,3532,25,142
2016-01-01,NY,2,3532,25,142
2016-01-01,TX,1,2532,45,420
2016-01-01,TX,2,2532,45,234
2016-01-02,CA,1,1354,53,231
2016-01-02,CA,2,1354,53,234
2016-01-02,MN,None,2366,85,None


## Question 4
In your opinion, what is the most efficient store and why?​

Suppose we are business owners, it is reasonable that we care about the profit those stores can make. The profit depends on the revenue and the expense(for example, employees' salaries), where profit = revenue - expense. If the store can make the most profit in a certain period of time, then we consider it the most efficient store.

## Question 5
(Challenge) Generate a query to rank in order the top 10 revenue producing states​.

While there will only be 3 states in the output, the SQL statement allows for the top 10 to be displayed in the hypothetical case that more states are added to the dataset in the future.


We will calculate the total amount of revenue by each state. And rank them in descending order so that the state with the higher revenue comes first. Then we select the top 10 states. 

Since we only want to display the states, we will use common table expression again to generate a temporary table. And select only states from that temporary table.

In [14]:
%%sql

WITH cte AS(
SELECT store_location, SUM(revenue) AS sum_revenue
FROM store_revenue
GROUP BY store_location
ORDER BY sum_revenue DESC
LIMIT 10
)

SELECT store_location
FROM cte

 * sqlite:///assessment.db
Done.


store_location
United States-CA
United States-NY
United States-TX
